In [13]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [14]:
#!pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple some-package

In [15]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [65]:
y_eval.value_counts()

0    165
1     99
Name: survived, dtype: int64

In [16]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [17]:
import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0],True)
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

def one_hot_cat_column(feature_name, vocab):
  return fc.indicator_column(
      fc.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))

In [18]:
## Preprocess the data
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Need to one-hot encode categorical features.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(fc.numeric_column(feature_name,
                                           dtype=tf.float32))

In [19]:
# Use entire batch since this is such a small dataset.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # For training, cycle thru dataset as many times as need (n_epochs=None).
    dataset = (dataset
      .repeat(n_epochs)
      .batch(NUM_EXAMPLES))
    return dataset
  return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)
#eval_input_fn = make_input_fn(dfeval, y_eval)

In [80]:

config = tf.estimator.RunConfig(model_dir='/home/luoyuhao/tmp/save',
                               keep_checkpoint_max=2)
head = tf.estimator.RegressionHead(label_dimension=1) 
est = tf.estimator.BoostedTreesEstimator(feature_columns=feature_columns,
                                          n_batches_per_layer=1, 
                                          head=head,
                                          n_trees=50, 
                                          max_depth=2,
                                          config=config)


INFO:tensorflow:Using config: {'_model_dir': '/home/luoyuhao/tmp/save', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f42acc7d4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Train model.
est.train(train_input_fn, max_steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 0 into /home/luoyuhao/tmp/save/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:loss = 0.3875598, step = 0


In [75]:
params = {
  'n_trees': 5,
  'max_depth': 1,
  'n_batches_per_layer': 1,
  # You must enable center_bias = True to get DFCs. This will force the model to
  # make an initial prediction before using any features (e.g. use the mean of
  # the training labels for regression or log odds for classification when
  # using cross entropy loss).
  'center_bias': True,
}

'''
__init__(
    feature_columns,
    n_batches_per_layer,
    model_dir=None,
    label_dimension=_HOLD_FOR_MULTI_DIM_SUPPORT,
    weight_column=None,
    n_trees=100,
    max_depth=6,
    learning_rate=0.1,
    l1_regularization=0.0,
    l2_regularization=0.0,
    tree_complexity=0.0,
    min_node_weight=0.0,
    config=None,
    center_bias=False,
    pruning_mode='none',
    quantile_sketch_epsilon=0.01,
    train_in_memory=False
)
'''
config = tf.estimator.RunConfig(model_dir='/home/luoyuhao/tmp/save',
                               keep_checkpoint_max=3,
                               save_checkpoints_steps=1000)

optimizer=lambda: tf.compat.v1.train.AdamOptimizer(
        learning_rate=tf.compat.v1.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf.compat.v1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.96))
adam_optim = tf.keras.optimizers.Adam(learning_rate=0.001) 
ws = tf.estimator.WarmStartSettings(ckpt_to_initialize_from="/home/luoyuhao/tmp/save")
est = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64,32,16,16],
    n_classes=2,
    config=config,
    optimizer=adam_optim,
    warm_start_from=ws) #
#est = tf.estimator.BoostedTreesClassifier(feature_columns,model_dir='/home/luoyuhao/tmp/save', **params)
#est = tf.estimator.LinearClassifier(feature_columns,model_dir='/home/luoyuhao/tmp/save')

INFO:tensorflow:Using config: {'_model_dir': '/home/luoyuhao/tmp/save', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f42ceab4550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [76]:
# Train model.
est.train(train_input_fn, max_steps=4000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/home/luoyuhao/tmp/save', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /home/luoyuhao/tmp/save
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/luoyuhao/tmp/save/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2000 into /home/luoyuhao/tmp/save/model.ckpt.
INFO:tensorflow:loss = 0.14331841, step = 2000
INFO:tensorflow:global_step/sec: 121.47
INFO:tensorflow:loss = 0.13937348, step = 2100 (0.824 sec)
INFO:tensorflow:global_step/sec: 132.868
INFO:tensorflow:loss = 0.1393839

In [77]:
# Evaluation.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

,0
accuracy,0.772727
accuracy_baseline,0.625000
auc,0.768320
auc_precision_recall,0.646437
average_loss,2.868385
label/mean,0.375000
loss,2.868385
precision,0.675676
prediction/mean,0.445424
recall,0.757576


In [71]:
results

{'accuracy': 0.7689394,
 'accuracy_baseline': 0.625,
 'auc': 0.7955617,
 'auc_precision_recall': 0.67886615,
 'average_loss': 1.2115315,
 'label/mean': 0.375,
 'loss': 1.2115315,
 'precision': 0.66101694,
 'prediction/mean': 0.45572412,
 'recall': 0.7878788,
 'global_step': 2000}

In [ ]:
## Model interpretation and plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns_colors = sns.color_palette('colorblind')

In [ ]:
'''
example = dftrain.head(1)
class_fc = one_hot_cat_column('class', ('First', 'Second', 'Third'))
print('Feature value: "{}"'.format(example['class'].iloc[0]))
print('One-hot encoded: ', fc.input_layer(dict(example), [class_fc]).numpy())
'''
